Goal: pull out buoy records that have turbidity

Order of operations:
- load all_relevant_stations.pkl
- sort by chl|turb|sedi|par|phot|secchi|ssc
- sort those by buoy/glider/ship

In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import urllib.request
import json 
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [9]:
# load a list of all the relevant ERDDAPs and their urls.
FF = pd.read_pickle("all_relevant_stations.pkl")

In [12]:
FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])
FF = FF.drop(columns="check_sum")

In [13]:
FF = pd.read_pickle("all_relevant_stations.pkl")

,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units
0,delayed_SEA068_M27,https://erddap.observations.voiceoftheocean.or...,2022-07-27T17:09:12Z,2022-07-31T06:39:19Z,58.199730,58.266098,19.885965,19.984983,degrees_north,degrees_east
1,nrt_SEA068_M27,https://erddap.observations.voiceoftheocean.or...,2022-07-27T17:09:41Z,2022-07-31T03:51:42Z,58.199850,58.265983,19.901883,19.982300,degrees_north,degrees_east
2,ctd_deployment,https://erddap.observations.voiceoftheocean.or...,2023-03-18T10:15:18Z,2024-06-18T11:20:53Z,55.250000,60.048000,10.853000,20.053000,degrees_north,degrees_east
3,delayed_SEA076_M8,https://erddap.observations.voiceoftheocean.or...,2022-10-05T15:03:41Z,2022-10-19T22:12:34Z,55.532133,55.746350,15.736800,16.323967,degrees_north,degrees_east
4,nrt_SEA076_M8,https://erddap.observations.voiceoftheocean.or...,2022-10-05T15:04:10Z,2022-10-19T20:25:55Z,55.532217,55.746117,15.736967,16.323750,degrees_north,degrees_east
5,delayed_SEA076_M9,https://erddap.observations.voiceoftheocean.or...,2022-10-20T15:11:16Z,2022-10-31T14:23:39Z,55.541417,55.661467,15.739717,16.296767,degrees_north,degrees_east
6,nrt_SEA076_M9,https://erddap.observations.voiceoftheocean.or...,2022-10-20T15:11:45Z,2022-10-31T13:05:46Z,55.544550,55.661333,15.739817,16.296533,degrees_north,degrees_east
7,delayed_SEA076_M13,https://erddap.observations.voiceoftheocean.or...,2023-01-13T02:09:07Z,2023-02-14T11:07:08Z,58.250300,58.517750,19.400867,20.098333,degrees_north,degrees_east
8,nrt_SEA076_M13,https://erddap.observations.voiceoftheocean.or...,2023-01-13T02:09:24Z,2023-02-14T10:34:35Z,58.250300,58.517300,19.400867,20.098333,degrees_north,degrees_east
9,delayed_SEA076_M16,https://erddap.observations.voiceoftheocean.or...,2023-04-11T12:59:37Z,2023-05-16T12:00:57Z,57.992133,58.514050,19.785217,19.967433,degrees_north,degrees_east


In [4]:
# "chl|turb|sedi|par|phot|secchi|ssc"
FF["buoy"] = False
FF["glider"] = False
FF["ship"] = False
FF["turbid"] = False
FF["ssc"] = False
FF["phot"] = False
FF["chl"] = False
FF['bad'] = False
lib = {'buoy','glider','ship','turbid','ssc','phot','chl'}


for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity

    # make the info URL for this site for this time range
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv"
              )
    e.dataset_id = FF['sites'][jj]
    # e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
    try:
        ds_info = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"bad"] = True
    else:
        for kwd in lib:
            idx = ds_info["Value"].str.contains(kwd, flags = re.IGNORECASE)
            if any(idx==True)==True:
                FF.loc[jj,kwd] = True

FF = FF[FF.bad == False] 
FF.reset_index(drop=True, inplace=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


In [5]:
FF.to_pickle("labeled_relevant_stations.pkl")


In [8]:
FF[(FF.buoy == True) & (FF.ssc == True) & (FF.chl == True)]

,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units,check_sum,buoy,glider,ship,turb,ssc,phot,chl,bad
874,URI_168-MV_BottomSonde,http://www.neracoos.org/erddap/,2006-05-17T11:45:01Z,2020-11-10T10:00:15Z,41.63825,41.63825,-71.38382,-71.38382,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
875,URI_169-QP_BottomSonde,http://www.neracoos.org/erddap/,2006-05-18T11:45:01Z,2020-11-10T11:00:00Z,41.5875,41.5875,-71.37992,-71.37992,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
876,URI_169-QP_SurfaceSonde,http://www.neracoos.org/erddap/,2006-05-18T11:45:01Z,2020-11-10T11:00:00Z,41.5875,41.5875,-71.37992,-71.37992,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
878,URI_CPoint_SurfaceSonde,http://www.neracoos.org/erddap/,2019-07-16T08:00:00Z,2020-10-22T00:45:00Z,41.700214,41.700214,-71.333506,-71.333506,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
879,URI_GBay_BottomSonde,http://www.neracoos.org/erddap/,2018-06-14T21:15:00Z,2020-05-29T05:15:00Z,41.68607,41.68607,-71.44592,-71.44592,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
880,URI_GBay_SurfaceSonde,http://www.neracoos.org/erddap/,2018-06-14T21:15:00Z,2019-07-09T13:00:00Z,41.68607,41.68607,-71.44592,-71.44592,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
881,URI_GSOPier_ExoTop,http://www.neracoos.org/erddap/,2017-05-30T19:30:00Z,2020-10-07T15:30:00Z,41.49216,41.49216,-71.371187,-71.371187,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
883,URI_NP_SurfaceSonde,http://www.neracoos.org/erddap/,2019-07-16T08:00:00Z,2019-09-17T13:00:00Z,41.666735,41.666735,-71.350082,-71.350082,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
885,URI_PP_SurfaceSonde,http://www.neracoos.org/erddap/,2019-07-16T08:00:00Z,2020-10-08T12:00:00Z,41.633598,41.633598,-71.316691,-71.316691,degrees_north,degrees_east,1.0,True,False,False,False,True,False,True,False
